##### Jupyter Notebook - GuardDuty Findings Analysis
This notebook is used to conduct Amazon GuardDuty findings analysis.The notebook will guide you in connecting and quering Amazon Athena databases and tables. 

**Authors: Tim Manik & Jeremy Wade**

*Note: This notebook assumes you have set up Amazon Security Lake within your AWS environment. You can replace the database and table names to match your own setup.*

# Setup

## Load Libraries

In order to query CloudTrail and interact with AWS, we need to load several libraries and configure our environment.

In [ ]:
pip install pyathena --quiet

In [ ]:
import boto3  # the Python SDK for AWS
from datetime import datetime
import pandas as pd # Pandas is a data analysis tool for Python
from pyathena import connect # Python API client for Amazon Athena
import json
region='us-east-1' # Set region variable for API commands. For example: us-east-1
athena_bucket = f's3://aws-athena-query-results-123456789123-{region}'  # S3 bucket that is configured to store your Athena queries
db_name = 'amazon_security_lake_glue_db_us_east_1' # database used by Athena. Choose 'default' if your CloudTrail was created via the Athena console and 'security_analysis' if you are using the AWS Security Analytics Bootstrap

Note: The following variables will need to be set to the aproriate values prior to running the cell: region, athena-bucket, db_name. 

## 1.0 Set up helper function for Athena

The Python query_results function shown below will help you query Athena tables.

In [ ]:
def query_results(sql):
    
    cursor = connect(s3_staging_dir=athena_bucket, region_name=region).cursor()
    cursor.execute(sql)
    
    columns = cursor.description
    data = cursor.fetchall()

    column_names = [column[0] for column in columns]
    rows = [dict(zip(column_names, data_row)) for data_row in data]

    df = pd.DataFrame(rows, columns=column_names)
    df1 = df.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    
    return df1.set_properties(**{'text-align': 'center'})

# 2.0 Investigate GuardDuty findings

### 2.1 Show GuardDuty findings
List GuardDuty findings via Security Hub logs to get a broad view of the types of logs we will be working with

In [ ]:
broad_gd = f"""
SELECT * 
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_sh_findings_1_0" 
WHERE metadata.product.feature.name = 'GuardDuty'
limit 5;
"""

In [ ]:
results = query_results(broad_gd)
results

List the amount of critical, high, medium, and low severity GuardDuty findings

In [ ]:
severity_count = f"""
SELECT
count(*) as count, severity as severity
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_sh_findings_1_0" 
WHERE metadata.product.feature.name = 'GuardDuty'
GROUP BY severity
limit 5;
"""

In [ ]:
results = query_results(severity_count)
results

List most common high severity GuardDuty finding by finding title (type)

In [ ]:
sev_findings_count = f"""
SELECT
count(*) as count, severity as severity, finding.title as findingTitle
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_sh_findings_1_0" 
WHERE metadata.product.feature.name = 'GuardDuty'
AND severity = 'High'
GROUP BY severity, finding.title
ORDER BY count DESC
limit 10;
"""

In [ ]:
results = query_results(sev_findings_count)
results

List most common resources associated with high severity GuardDuty findings

In [ ]:
highsev_resources = f"""
SELECT
count(*) as count, severity as severity, resources[1].type as resourceType, resources[1].uid as resourceId, finding.title as findingTitle
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_sh_findings_1_0" 
WHERE metadata.product.feature.name = 'GuardDuty'
AND severity = 'High'
GROUP BY severity, resources[1].type, resources[1].uid, finding.title
ORDER BY count DESC
limit 10;
"""

In [ ]:
results = query_results(highsev_resources)
results

Show all GuardDuty findings against a specific resource

In [ ]:
resource_arn = 'ARN'

Note: The resource_arn variable needs to be set to the ARn of the AWS resource you want to investigate. 

In [ ]:

specific_resource = f"""
SELECT
finding.first_seen_time FirstSeen, severity as severity, resources[1].type as resourceType, resources[1].uid as resourceId, finding.title as findingTitle
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_sh_findings_1_0" 
WHERE metadata.product.feature.name = 'GuardDuty'
AND severity = 'High'
AND resources[1].uid = '{resource_arn}'
ORDER BY finding.first_seen_time ASC
limit 20;
"""

In [ ]:
results = query_results(specific_resource)
results

In [ ]:


specific_resource = f"""
SELECT
finding.first_seen_time FirstSeen, count(finding.first_seen_time), severity as severity, resources[1].type as resourceType, resources[1].uid as resourceId, finding.title as findingTitle
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_sh_findings_1_0" 
WHERE metadata.product.feature.name = 'GuardDuty'
AND severity = 'High'
AND resources[1].uid = '{resource_arn}'
GROUP BY finding.title, finding.first_seen_time, severity, resources[1].type, resources[1].uid
ORDER BY finding.first_seen_time ASC
limit 20;
"""

In [ ]:
results = query_results(specific_resource)
results